In [1]:
import warnings
warnings.filterwarnings(action="ignore")

import numpy as np
from os import listdir
from matplotlib import pyplot as plt
from tensorflow.python.keras.preprocessing.image import load_img,img_to_array
from tensorflow.python.keras.initializers import RandomNormal
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.layers import Input,Concatenate
from tensorflow.python.keras.layers import Conv2D,LeakyReLU
from tensorflow.python.keras.layers import BatchNormalization,Dropout
from tensorflow.python.keras.layers import Activation,Conv2DTranspose
from tensorflow.python.keras.models import Model,load_model

# Model

### Discriminator

In [2]:
def define_discriminator(image_shape):
    # weight intialization
    init = RandomNormal(stddev = 0.02)
    #source image input
    in_src_image = Input(shape = image_shape)
    #target image_input
    in_target_image = Input(shape = image_shape)
    #concatenate images channel-wise
    merged = Concatenate()([in_src_image,in_target_image])
    #C64
    d = Conv2D(64,(4,4),strides=(2,2),padding="same",kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    #C128
    d = Conv2D(128,(4,4),strides=(2,2),padding="same",kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    #C256
    d = Conv2D(256,(4,4),strides=(2,2),padding="same",kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    #C512
    d = Conv2D(512,(4,4),strides=(2,2),padding="same",kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    #second Last Output Layer
    d = Conv2D(512,(4,4),padding="same",kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    #Patch Output
    d = Conv2D(1,(4,4),padding="same",kernel_initializer=init)(d)
    patch_out = Activation("sigmoid")(d)
    #define Model
    model = Model([in_src_image,in_target_image],patch_out)
    opt = Adam(lr=0.0002,beta_1=0.5)
    model.compile(loss="binary_crossentropy",optimizer=opt,loss_weights=[0.5])
    return model

In [3]:
# Define an encoder block
def define_encoder_block(layer_in,n_filters,batchnorm=True):
    #weight intialization
    init = RandomNormal(stddev=0.02)
    g = Conv2D(n_filters,(4,4),strides=(2,2),padding='same',kernel_initializer=init)(layer_in)
    #add batchnorm conditionally
    if batchnorm:
        g=BatchNormalization()(g,training=True)
    # leaky Relu activation Function
    g = LeakyReLU(alpha=0.2)(g)
    return g

def decoder_block(layer_in,skip_in,n_filters,dropout=True):
    #Weight intialization
    init = RandomNormal(stddev=0.02)
    g = Conv2DTranspose(n_filters,(4,4),strides=(2,2),padding="same",kernel_initializer=init)(layer_in)
    g = BatchNormalization()(g,training=True)
    # Conditionally add Dropout
    if dropout:
        g = Dropout(0.5)(g,training=True)
    g = Concatenate()([g,skip_in])
    g = Activation("relu")(g)
    return g

def define_generator(image_shape=(256,256,3)):
    # weight intialization 
    init = RandomNormal(stddev=0.02)
    #image_input
    in_image = Input(shape = image_shape)
    #encoder model
    e1 = define_encoder_block(in_image,64,batchnorm=False)
    e2 = define_encoder_block(e1,128)
    e3 = define_encoder_block(e2,256)
    e4 = define_encoder_block(e3,512)
    e5 = define_encoder_block(e4,512)
    e6 = define_encoder_block(e5,512)
    e7 = define_encoder_block(e6,512)
    #bottleneck ,no batch norm and relu
    b = Conv2D(512,(4,4),strides=(2,2),padding="same",kernel_initializer=init)(e7)
    b = Activation("relu")(b)
    #decoder model
    d1 = decoder_block(b,e7,512)
    d2 = decoder_block(d1,e6,512)
    d3 = decoder_block(d2,e5,512)
    d4 = decoder_block(d3,e4,512,dropout=False)
    d5 = decoder_block(d4,e3,256,dropout=False)
    d6 = decoder_block(d5,e2,128,dropout=False)
    d7 = decoder_block(d6,e1,512,dropout=False)
    #output
    g = Conv2DTranspose(3,(4,4),strides=(2,2),padding="same",kernel_initializer=init)(d7)
    out_image = Activation("tanh")(g)
    #define model
    model = Model(in_image,out_image)
    return model

In [4]:
def define_gan(g_model,d_model,image_shape):
    #make weights in the discriminator not trainable
    d_model.trainable=False
    #define the source image
    in_src = Input(shape=image_shape)
    #connect the source image to the generator input
    gen_out = g_model(in_src)
    #connect the source input ans generator output to the discriminator input
    dis_out = d_model([in_src,gen_out])
    model = Model(in_src, [dis_out, gen_out])
    #compile model
    opt = Adam(lr=  0.0002,beta_1=0.5)
    model.compile(loss=['binary_crossentropy','mae'],optimizer=opt,loss_weights=[1,100])
    return model

In [5]:
# load and prepare training images
def load_real_samples(filename):
    # load compressed arrays
    data = np.load(filename)
    # unpack arrays
    X1, X2 = data['arr_0'], data['arr_1']
    # scale from [0,255] to [-1,1]
    X1 = (X1 - 127.5) / 127.5
    X2 = (X2 - 127.5) / 127.5
    return [X1, X2]

In [6]:
#select a batch of random samples, returns images and target
def generate_real_samples(dataset,n_samples,patch_shape):
    #unpack dataset
    trainA,trainB = dataset
    #choose random instances 
    ix = np.random.randint(0,trainA.shape[0],n_samples)
    #retrive selected images
    X1,X2 = trainA[ix],trainB[ix]
    #generate "real" class labels(1)
    y = np.ones((n_samples,patch_shape,patch_shape,1))
    return [X1,X2],y

In [7]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, patch_shape):
    # generate fake instance
    X = g_model.predict(samples)
    # create 'fake' class labels (0)
    y = np.zeros((len(X), patch_shape, patch_shape, 1))
    return X, y

In [8]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, dataset, n_samples=3):
    # select a sample of input images
    [X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
    # generate a batch of fake samples
    X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
    # scale all pixels from [-1,1] to [0,1]
    X_realA = (X_realA + 1) / 2.0
    X_realB = (X_realB + 1) / 2.0
    X_fakeB = (X_fakeB + 1) / 2.0
    # plot real source images
    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + i)
        plt.axis('off')
        plt.imshow(X_realA[i])
    # plot generated target image
    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + n_samples + i)
        plt.axis('off')
        plt.imshow(X_fakeB[i])
    # plot real target image
    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + n_samples*2 + i)
        plt.axis('off')
        plt.imshow(X_realB[i])
    # save plot to file
    filename1 = 'plot_%06d.png' % (step+1)
    plt.savefig(filename1)
    plt.close()
    # save the generator model
    filename2 = 'model_%06d.h5' % (step+1)
    g_model.save(filename2)
    print('>Saved: %s and %s' % (filename1, filename2))

In [9]:
# train pix2pix model
def train(d_model, g_model, gan_model, dataset, n_epochs=500, n_batch=1):
    # determine the output square shape of the discriminator
    n_patch = d_model.output_shape[1]
    # unpack dataset
    trainA, trainB = dataset
    # calculate the number of batches per training epoch
    bat_per_epo = int(len(trainA) / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    # manually enumerate epochs
    for i in range(n_steps):
        # select a batch of real samples
        [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
        # generate a batch of fake samples
        X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
        # update discriminator for real samples
        d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
        # update discriminator for generated samples
        d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
        # update the generator
        g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
        # summarize performance
        print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
        # summarize model performance
        if (i+1) % (bat_per_epo * 10) == 0:
            summarize_performance(i, g_model, dataset)

In [10]:
# load image data
dataset = load_real_samples('data/mask_new_256.npz')
print('Loaded', dataset[0].shape, dataset[1].shape)
# define input shape based on the loaded dataset
image_shape = dataset[0].shape[1:]
# define the models
d_model = define_discriminator(image_shape)
g_model = define_generator(image_shape)
# define the composite model
gan_model = define_gan(g_model, d_model, image_shape)
# train model
train(d_model, g_model, gan_model, dataset)

Loaded (50, 256, 256, 3) (50, 256, 256, 3)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
>1, d1[0.298] d2[1.168] g[77.710]
>2, d1[0.534] d2[0.966] g[48.836]
>3, d1[0.366] d2[0.735] g[38.970]
>4, d1[0.389] d2[0.577] g[32.745]
>5, d1[0.431] d2[0.590] g[31.127]
>6, d1[0.543] d2[0.560] g[47.723]
>7, d1[0.432] d2[0.580] g[31.692]
>8, d1[0.396] d2[0.482] g[27.908]
>9, d1[0.418] d2[0.417] g[30.005]
>10, d1[0.388] d2[0.505] g[33.327]
>11, d1[0.434] d2[0.467] g[45.573]
>12, d1[0.470] d2[0.456] g[30.723]
>13, d1[0.409] d2[0.416] g[21.367]
>14, d1[0.461] d2[0.477] g[24.306]
>15, d1[0.398] d2[0.394] g[19.174]
>16, d1[0.376] d2[0.378] g[29.705]
>17, d1[0.389] d2[0.391] g[28.559]
>18, d1[0.460] d2[0.443] g[23.060]
>19, d1[0.371] d2[0.431] g[23.299]
>20, d1[0.432] d2[0.483] g[27.247]
>21, d1[0.442] d2[0.415] g[20.748]
>22, d1[0.404] d2[0.358] g[27.606]
>23, d1[0.377] d2[0.437] g[41.766]
>24, d1[0.406] d2[0.408] g[28.701]
>25, d1[

>221, d1[0.986] d2[0.452] g[13.597]
>222, d1[0.293] d2[0.247] g[22.106]
>223, d1[0.495] d2[0.317] g[20.196]
>224, d1[0.182] d2[0.474] g[11.187]
>225, d1[0.325] d2[0.244] g[14.571]
>226, d1[0.351] d2[0.231] g[18.194]
>227, d1[0.196] d2[0.301] g[13.365]
>228, d1[0.525] d2[0.299] g[17.640]
>229, d1[0.121] d2[0.138] g[16.375]
>230, d1[0.079] d2[0.085] g[16.195]
>231, d1[0.155] d2[0.126] g[16.040]
>232, d1[0.031] d2[0.069] g[16.148]
>233, d1[0.198] d2[0.959] g[17.881]
>234, d1[0.142] d2[0.100] g[17.244]
>235, d1[0.251] d2[0.173] g[11.282]
>236, d1[0.142] d2[0.266] g[15.101]
>237, d1[0.152] d2[0.052] g[15.773]
>238, d1[1.500] d2[0.098] g[10.888]
>239, d1[0.799] d2[0.337] g[10.147]
>240, d1[0.664] d2[0.253] g[27.172]
>241, d1[0.120] d2[0.292] g[14.470]
>242, d1[0.021] d2[0.676] g[20.034]
>243, d1[0.251] d2[0.130] g[14.569]
>244, d1[0.180] d2[0.119] g[19.813]
>245, d1[0.038] d2[0.067] g[16.104]
>246, d1[0.183] d2[0.688] g[13.126]
>247, d1[0.914] d2[0.135] g[15.352]
>248, d1[0.333] d2[0.232] g[

>449, d1[1.017] d2[0.338] g[11.381]
>450, d1[0.301] d2[0.200] g[11.755]
>451, d1[0.038] d2[0.095] g[11.884]
>452, d1[0.090] d2[0.032] g[21.423]
>453, d1[0.059] d2[0.330] g[18.974]
>454, d1[0.226] d2[0.086] g[14.142]
>455, d1[0.485] d2[0.645] g[17.898]
>456, d1[0.200] d2[0.097] g[17.070]
>457, d1[0.050] d2[0.074] g[13.122]
>458, d1[0.841] d2[1.483] g[10.795]
>459, d1[0.016] d2[0.378] g[11.046]
>460, d1[0.074] d2[0.052] g[11.086]
>461, d1[0.035] d2[0.173] g[24.586]
>462, d1[0.169] d2[0.034] g[10.827]
>463, d1[0.153] d2[0.054] g[11.386]
>464, d1[0.346] d2[0.383] g[14.543]
>465, d1[0.013] d2[0.087] g[10.943]
>466, d1[0.172] d2[0.059] g[14.750]
>467, d1[0.030] d2[0.031] g[15.643]
>468, d1[0.016] d2[0.034] g[10.154]
>469, d1[0.048] d2[0.042] g[16.633]
>470, d1[0.068] d2[0.049] g[26.575]
>471, d1[0.158] d2[0.046] g[13.184]
>472, d1[0.017] d2[0.222] g[13.531]
>473, d1[0.030] d2[0.027] g[16.526]
>474, d1[0.016] d2[0.022] g[15.598]
>475, d1[0.176] d2[0.137] g[10.075]
>476, d1[0.474] d2[0.154] g[

>677, d1[0.021] d2[0.030] g[6.243]
>678, d1[0.269] d2[0.141] g[9.675]
>679, d1[0.031] d2[0.248] g[9.159]
>680, d1[0.287] d2[0.101] g[11.581]
>681, d1[0.034] d2[0.018] g[8.503]
>682, d1[0.311] d2[0.114] g[8.429]
>683, d1[0.012] d2[0.130] g[10.238]
>684, d1[0.006] d2[0.283] g[15.151]
>685, d1[0.081] d2[0.009] g[11.363]
>686, d1[0.090] d2[0.014] g[9.177]
>687, d1[0.116] d2[0.125] g[13.718]
>688, d1[0.018] d2[0.070] g[11.125]
>689, d1[0.056] d2[0.013] g[11.888]
>690, d1[0.055] d2[0.049] g[11.260]
>691, d1[0.031] d2[0.020] g[12.132]
>692, d1[0.030] d2[0.008] g[10.742]
>693, d1[0.013] d2[0.032] g[12.822]
>694, d1[0.007] d2[0.008] g[7.656]
>695, d1[0.015] d2[0.007] g[11.682]
>696, d1[0.012] d2[0.007] g[10.702]
>697, d1[0.066] d2[0.025] g[11.303]
>698, d1[0.016] d2[0.026] g[9.346]
>699, d1[0.006] d2[0.020] g[13.834]
>700, d1[0.003] d2[0.009] g[11.888]
>701, d1[0.014] d2[0.009] g[9.968]
>702, d1[0.028] d2[0.009] g[7.914]
>703, d1[0.012] d2[0.013] g[6.558]
>704, d1[0.025] d2[0.006] g[9.786]
>705

>908, d1[0.006] d2[0.003] g[16.777]
>909, d1[0.008] d2[0.006] g[10.251]
>910, d1[0.002] d2[0.046] g[10.714]
>911, d1[0.002] d2[0.002] g[8.842]
>912, d1[0.043] d2[0.001] g[11.849]
>913, d1[0.007] d2[0.005] g[6.066]
>914, d1[0.002] d2[0.003] g[9.141]
>915, d1[0.002] d2[0.003] g[11.847]
>916, d1[0.008] d2[0.013] g[7.882]
>917, d1[0.002] d2[0.010] g[8.689]
>918, d1[0.001] d2[0.004] g[12.831]
>919, d1[0.003] d2[0.003] g[10.069]
>920, d1[0.001] d2[0.002] g[8.958]
>921, d1[0.002] d2[0.002] g[8.193]
>922, d1[0.025] d2[0.002] g[9.903]
>923, d1[0.001] d2[0.021] g[8.161]
>924, d1[0.001] d2[0.002] g[6.847]
>925, d1[0.008] d2[0.005] g[10.851]
>926, d1[0.001] d2[0.002] g[7.880]
>927, d1[0.003] d2[0.002] g[10.288]
>928, d1[0.006] d2[0.004] g[11.102]
>929, d1[0.007] d2[0.003] g[7.934]
>930, d1[0.002] d2[0.003] g[9.393]
>931, d1[0.002] d2[0.003] g[8.799]
>932, d1[0.001] d2[0.006] g[9.462]
>933, d1[0.002] d2[0.003] g[8.166]
>934, d1[0.005] d2[0.004] g[9.985]
>935, d1[0.060] d2[0.019] g[10.815]
>936, d1[

...

>14210, d1[0.000] d2[0.000] g[3.187]
>14211, d1[0.000] d2[0.000] g[2.878]
>14212, d1[0.000] d2[0.000] g[2.647]
>14213, d1[0.000] d2[0.000] g[2.388]
>14214, d1[0.000] d2[0.000] g[3.216]
>14215, d1[0.000] d2[0.000] g[2.724]
>14216, d1[0.000] d2[0.000] g[2.471]
>14217, d1[0.000] d2[0.000] g[1.485]
>14218, d1[0.000] d2[0.002] g[3.176]
>14219, d1[0.000] d2[0.000] g[1.566]
>14220, d1[0.000] d2[0.000] g[2.230]
>14221, d1[0.000] d2[0.000] g[2.379]
>14222, d1[0.001] d2[0.000] g[1.901]
>14223, d1[0.000] d2[0.000] g[1.883]
>14224, d1[0.000] d2[0.000] g[2.062]
>14225, d1[0.001] d2[0.001] g[3.278]
>14226, d1[0.000] d2[0.000] g[2.311]
>14227, d1[0.000] d2[0.000] g[3.153]
>14228, d1[0.000] d2[0.000] g[2.426]
>14229, d1[0.000] d2[0.000] g[2.262]
>14230, d1[0.000] d2[0.000] g[1.988]
>14231, d1[0.000] d2[0.000] g[2.070]
>14232, d1[0.000] d2[0.000] g[2.668]
>14233, d1[0.000] d2[0.000] g[2.934]
>14234, d1[0.001] d2[0.001] g[3.202]
>14235, d1[0.000] d2[0.000] g[2.630]
>14236, d1[0.000] d2[0.000] g[2.478]
>

>14432, d1[0.000] d2[0.000] g[2.667]
>14433, d1[0.000] d2[0.000] g[2.754]
>14434, d1[0.000] d2[0.000] g[2.157]
>14435, d1[0.000] d2[0.000] g[2.452]
>14436, d1[0.000] d2[0.001] g[3.195]
>14437, d1[0.000] d2[0.000] g[2.467]
>14438, d1[0.000] d2[0.000] g[3.028]
>14439, d1[0.000] d2[0.000] g[2.150]
>14440, d1[0.001] d2[0.001] g[1.397]
>14441, d1[0.000] d2[0.000] g[2.135]
>14442, d1[0.001] d2[0.000] g[3.319]
>14443, d1[0.001] d2[0.000] g[1.773]
>14444, d1[0.000] d2[0.000] g[2.833]
>14445, d1[0.000] d2[0.000] g[1.866]
>14446, d1[0.000] d2[0.000] g[1.884]
>14447, d1[0.000] d2[0.000] g[1.580]
>14448, d1[0.000] d2[0.000] g[2.914]
>14449, d1[0.001] d2[0.000] g[2.325]
>14450, d1[0.000] d2[0.000] g[2.031]
>14451, d1[0.000] d2[0.000] g[2.612]
>14452, d1[0.001] d2[0.000] g[2.757]
>14453, d1[0.000] d2[0.000] g[2.086]
>14454, d1[0.000] d2[0.001] g[3.195]
>14455, d1[0.000] d2[0.000] g[2.527]
>14456, d1[0.000] d2[0.000] g[2.207]
>14457, d1[0.001] d2[0.000] g[1.929]
>14458, d1[0.000] d2[0.000] g[2.422]
>

>14653, d1[0.000] d2[0.000] g[2.727]
>14654, d1[0.000] d2[0.000] g[1.547]
>14655, d1[0.000] d2[0.000] g[3.046]
>14656, d1[0.000] d2[0.000] g[2.999]
>14657, d1[0.000] d2[0.000] g[2.278]
>14658, d1[0.000] d2[0.000] g[1.641]
>14659, d1[0.000] d2[0.000] g[3.303]
>14660, d1[0.000] d2[0.000] g[3.225]
>14661, d1[0.000] d2[0.000] g[2.778]
>14662, d1[0.000] d2[0.001] g[3.151]
>14663, d1[0.000] d2[0.000] g[2.121]
>14664, d1[0.000] d2[0.000] g[2.533]
>14665, d1[0.001] d2[0.000] g[2.836]
>14666, d1[0.000] d2[0.000] g[2.181]
>14667, d1[0.000] d2[0.000] g[2.158]
>14668, d1[0.000] d2[0.000] g[1.550]
>14669, d1[0.001] d2[0.000] g[1.620]
>14670, d1[0.000] d2[0.000] g[2.747]
>14671, d1[0.000] d2[0.000] g[2.493]
>14672, d1[0.000] d2[0.000] g[2.789]
>14673, d1[0.000] d2[0.000] g[1.901]
>14674, d1[0.000] d2[0.000] g[1.861]
>14675, d1[0.000] d2[0.000] g[2.272]
>14676, d1[0.000] d2[0.000] g[1.559]
>14677, d1[0.000] d2[0.000] g[2.417]
>14678, d1[0.000] d2[0.000] g[4.265]
>14679, d1[0.000] d2[0.000] g[2.688]
>

>14875, d1[0.000] d2[0.000] g[2.285]
>14876, d1[0.000] d2[0.000] g[2.140]
>14877, d1[0.001] d2[0.000] g[2.361]
>14878, d1[0.000] d2[0.000] g[2.802]
>14879, d1[0.000] d2[0.000] g[1.797]
>14880, d1[0.000] d2[0.000] g[2.412]
>14881, d1[0.000] d2[0.000] g[2.588]
>14882, d1[0.000] d2[0.000] g[1.574]
>14883, d1[0.000] d2[0.000] g[1.938]
>14884, d1[0.000] d2[0.000] g[2.939]
>14885, d1[0.000] d2[0.000] g[2.769]
>14886, d1[0.000] d2[0.000] g[2.933]
>14887, d1[0.000] d2[0.000] g[1.541]
>14888, d1[0.000] d2[0.000] g[2.228]
>14889, d1[0.001] d2[0.000] g[2.996]
>14890, d1[0.000] d2[0.000] g[2.775]
>14891, d1[0.000] d2[0.000] g[2.182]
>14892, d1[0.000] d2[0.000] g[2.993]
>14893, d1[0.000] d2[0.000] g[2.233]
>14894, d1[0.000] d2[0.000] g[2.244]
>14895, d1[0.000] d2[0.000] g[1.697]
>14896, d1[0.000] d2[0.000] g[2.787]
>14897, d1[0.000] d2[0.000] g[3.011]
>14898, d1[0.001] d2[0.000] g[2.276]
>14899, d1[0.000] d2[0.000] g[2.331]
>14900, d1[0.000] d2[0.000] g[2.247]
>14901, d1[0.000] d2[0.000] g[1.558]
>

KeyboardInterrupt: 

In [70]:
train(d_model, g_model, gan_model, dataset)

>1, d1[0.000] d2[0.000] g[1.771]
>2, d1[0.000] d2[0.000] g[3.562]
>3, d1[0.000] d2[0.000] g[4.172]
>4, d1[0.000] d2[0.000] g[1.914]
>5, d1[0.000] d2[0.000] g[2.031]
>6, d1[0.000] d2[0.000] g[1.836]


KeyboardInterrupt: 